In [18]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.ensemble import RandomForestClassifier
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gsaya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gsaya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gsaya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import gensim
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

In [10]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [19]:
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
			['this', 'is', 'the', 'second', 'sentence'],
			['yet', 'another', 'sentence'],
			['one', 'more', 'sentence'],
			['and', 'the', 'final', 'sentence']]

In [20]:
model = Word2Vec(sentences, size=150, min_count=1)

In [21]:
words = list(model.wv.vocab)

In [22]:
print(words)
# access vector for one word
print(len(model['sentence']))
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec', 'second', 'yet', 'another', 'one', 'more', 'and', 'final']
150
Word2Vec(vocab=14, size=150, alpha=0.025)


C:\Users\gsaya\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
train_data = pd.read_csv('training_backup.csv')
test_data = pd.read_csv('testing_backup.csv')

In [24]:
train_data.head()

,lyrics,mood
0,Where was God when I needed a friend?\n\nAnd w...,3
1,We skipped the light Fandango\nTurned cartwhee...,2
2,(Everybody move to prove the groove)\n(Everybo...,1
3,Lo! The darkness has casted in parts.\r\nThat ...,1
4,One day when I came home at lunchtime I heard ...,1


In [38]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
# turn a doc into clean tokens
def clean_lyric(lyric):
    # split into tokens by white space
    tokens = lyric.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

# load doc and add to vocab
def add_lyric_to_vocab(lyrics, vocab):
    tokens = clean_lyric(lyrics)
    # update counts
    vocab.update(tokens)
    
# load all docs in a directory
def process_lyrics(lyric_df, vocab, is_trian):
    for lyric in lyric_df:
#         print(lyric)
        add_lyric_to_vocab(lyric, vocab)
        


In [42]:
from collections import Counter
vocab = Counter()
# add all docs to vocab
process_lyrics(train_data['lyrics'], vocab, True)
# print the size of the vocab
print(len(vocab))
# # print the top words in the vocab
print(vocab.most_common(50))

16033
[('And', 3685), ('Im', 3320), ('love', 2459), ('You', 2430), ('know', 2420), ('dont', 2003), ('like', 1851), ('But', 1510), ('go', 1503), ('never', 1374), ('The', 1366), ('get', 1302), ('one', 1253), ('time', 1226), ('got', 1168), ('see', 1156), ('Its', 1090), ('say', 1063), ('want', 1020), ('cant', 1015), ('back', 1008), ('away', 1007), ('way', 1001), ('youre', 991), ('let', 973), ('Oh', 945), ('So', 929), ('Ill', 919), ('heart', 910), ('come', 907), ('life', 857), ('day', 855), ('gonna', 845), ('feel', 832), ('We', 818), ('make', 808), ('take', 797), ('need', 787), ('baby', 775), ('Ive', 765), ('When', 756), ('Cause', 736), ('wanna', 727), ('oh', 710), ('If', 706), ('around', 701), ('world', 697), ('think', 686), ('My', 671), ('could', 670)]


In [45]:
min_occurane = 5
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

4428


In [84]:
# turn a doc into clean tokens
def Lyric_to_clean_lines(lyric, vocab):
    clean_lines = list()
    lyriclines = lyric.splitlines()
    for line in lyriclines:
        # split into tokens by white space
        if len(line) == 0:
            continue
        tokens = line.split()
        # remove punctuation from each token
        table = str.maketrans('', '', punctuation)
        tokens = [w.translate(table) for w in tokens]
        # filter out tokens not in vocab
        tokens = [w for w in tokens if w in vocab]
        clean_lines.append(tokens)
    return clean_lines

# load all docs in a directory
def Process_lyrics(lyric_df, vocab, is_trian):
    lines = list()
    for lyric in lyric_df:
#         print(lyric)
        lyric_lines = Lyric_to_clean_lines(lyric,vocab)
#         print(lyric_lines)
        if len(lyric_lines) > 0:
            lines += lyric_lines
    return lines

In [85]:
lines = Process_lyrics(train_data['lyrics'], vocab, True)

In [88]:
print(lines)
sentences = lines

[['Where', 'God', 'needed', 'friend'], ['And', 'God', 'came', 'end'], ['Where', 'God', 'lost', 'mind'], ['Where', 'God', 'find'], ['dont', 'wanna'], ['Dont', 'wanna'], ['Dont', 'wanna'], ['dont', 'wanna'], ['Where', 'love', 'felt', 'like', 'hate'], ['Where', 'hate', 'felt', 'like', 'love'], ['Where'], ['said', 'youd'], ['Where', 'fear'], ['said', 'scared'], ['dont', 'wanna'], ['Dont', 'wanna'], ['Dont', 'wanna'], ['dont', 'wanna'], ['Tell', 'believe'], ['Ill', 'tell', 'see'], ['Cause', 'dont', 'know', 'trust'], ['My', 'heart', 'filled', 'disgust'], ['cant', 'take'], ['Tell', 'believe'], ['Ill', 'tell', 'see'], ['Cause', 'dont', 'know', 'trust'], ['My', 'heart', 'filled', 'disgust'], ['Tell', 'believe'], ['Tell', 'believe'], ['Ladies', 'gentlemen'], ['May', 'attention'], ['Are', 'ready', 'joke'], ['Are', 'ready', 'great', 'deception'], ['Tell', 'believe'], ['Tell', 'believe'], ['Tell', 'believe'], ['Tell', 'believe'], ['Tell', 'believe'], ['Tell', 'believe'], ['We', 'skipped', 'light', 

In [89]:
model = Word2Vec(sentences, size=100, window=5, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 16033


In [101]:
# save model in ASCII (word2vec) format
filename = 'embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [131]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(sentences)

In [157]:
encoded_docs = tokenizer.texts_to_sequences(sentences)
max_length = 0
for s in sentences:
    max_length = max(max_length,len(s))
print(max_length)
from numpy import array
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define training labels
# print(Xtrain.shape)
ytrain = array([0 for _ in range(4563)] + [1 for _ in range(4563)])

96


In [158]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

# load embedding as a dict
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename, encoding="utf8")
	lines = file.readlines()[1:]
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
	return embedding

In [159]:
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


In [160]:
lines = Process_lyrics(test_data['lyrics'], vocab, True)

In [161]:
sentences = lines

In [162]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(sentences)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define test labels
print(Xtest.shape)
ytest = array([0 for _ in range(4563)] + [1 for _ in range(4563)])
print(ytest.shape)

(9126, 96)
(9126,)


In [163]:
vocab_size = len(tokenizer.word_index) + 1
 
# load embedding from file
raw_embedding = load_embedding('embedding_word2vec.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

In [164]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 96, 100)           299600    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 92, 128)           64128     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 46, 128)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 5888)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5889      
Total params: 369,617
Trainable params: 70,017
Non-trainable params: 299,600
_________________________________________________________________
None
Epoch 1/10
 - 9s - loss: 8.0348 - acc: 0.5000
Epoch 2/10
 - 7s - loss: 8.0590 - acc: 0.5000
Epoch 3/10
 - 8s - loss: 8.0590 - 